In [5]:
from kafka import KafkaProducer
import threading
import json
import random
import time

from equipo.random_tweets import generate_random_tweets  

KAFKA_SERVER = '7e9458d7d568:9093'

In [ ]:
def worker(user_id: int, topic: str, num_messages: int):

    # Initialize the Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_SERVER,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')  # serialize data as JSON
    )

    print(f"[Thread-{user_id}] Iniciando, topic={topic}, mensajes={num_messages}")
    
    # Produce data to Kafka topic
    try:
        for tweet in generate_random_tweets(num_messages, user_id):
            # Send data to Kafka
            producer.send(topic, tweet)
            print(f"[Thread-{user_id}] Sent to {topic}: {tweet['tweet_id']}")

            time.sleep(2)

    except KeyboardInterrupt:
        print(f"[Thread-{user_id}] Interrumpido por el usuario.")
        
    finally:
        producer.close()
        print(f"[Thread-{user_id}] Producer cerrado.")

In [7]:
if __name__ == "__main__":
    NUM_MESSAGES = random.randint(30, 50)
    threads = []

    for user_id in range(1, 5):
        topic = f"tweets-{user_id}"
        t = threading.Thread(
            target=worker,
            args=(user_id, topic, NUM_MESSAGES),
            name=f"Thread-{user_id}"
        )
        threads.append(t)
        t.start()

    # Espera a que todos los hilos terminen
    for t in threads:
        t.join()

    print("Todos los hilos han terminado.")


[Thread-1] Iniciando, topic=tweets-1, mensajes=49
[Thread-3] Iniciando, topic=tweets-3, mensajes=49
[Thread-2] Iniciando, topic=tweets-2, mensajes=49
[Thread-4] Iniciando, topic=tweets-4, mensajes=49
[Thread-3] Sent to tweets-3: f5f7cf09-09a5-4890-9740-63c7e1c43c45
[Thread-1] Sent to tweets-1: 5c4e6e99-3b9e-4527-b006-9f61ab274787
[Thread-2] Sent to tweets-2: 12e6f002-8fc3-46a2-9584-378632bed527
[Thread-4] Sent to tweets-4: b0650667-d4cb-4003-b241-a7c2e9d99ed3
[Thread-1] Sent to tweets-1: f2d4d076-2fa9-4015-bdd9-77a97129b826
[Thread-3] Sent to tweets-3: 0fe1abb3-e54d-4130-9b96-2800588133c8
[Thread-4] Sent to tweets-4: bf21fb98-d158-41e6-8cc6-8e25939b37aa
[Thread-2] Sent to tweets-2: ba079cac-e154-4be3-b309-07cd7e6ccc67
[Thread-1] Sent to tweets-1: 7150ace2-ad8e-4cc7-8567-1cbb67134aea
[Thread-3] Sent to tweets-3: 59790a7d-0392-4b55-a159-509814a569bf
[Thread-4] Sent to tweets-4: 7ddfa5be-d065-4fc2-9f89-279a50623d74
[Thread-3] Sent to tweets-3: 17c5b74b-d437-4be0-a351-1957342bfa80
[Thread-